Le but est de créer un jeu de données contenant les triplets généré à partir d'un ensemble de textes. 

- Source node , Target, Relationship label

Puisque je vais construire le KG avec py2neo, j'aurai donc besoin même des étiquettes dans lesquels source et target nodes appartient. 

La sortie de ce notebook est une dataset contenant les triplets et les étiquettes; Ce jeu de donnée sera ensuite utilisé pour créer le graphe de connaissances

In [1]:
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


In [2]:
!pip install torch==1.6.0
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.8/748.8 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0+cu116 requires torch==1.13.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.6.0 which is incompatible.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━

## **Let's Load our best ner & RE model**

In [3]:
import transformers
import torch
from transformers import BertForTokenClassification, AdamW
from transformers import BertTokenizer, BertConfig
import pandas as pd
from copy import deepcopy
from collections import OrderedDict
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
from nltk import tokenize

**Loading ner model**

In [4]:
# If the tokenizer uses a single vocabulary file, you can point directly to this file
tokenizer = AutoTokenizer.from_pretrained('/drive/My Drive/bestmodel/')
model = AutoModelForTokenClassification.from_pretrained('/drive/My Drive/bestmodel/', ignore_mismatched_sizes=True)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at /drive/My Drive/bestmodel/ and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
id2label= {
    "-100": "IGN",
    "0": "B-DISEASE",
    "1": "I-DISEASE",
    "2": "B-RAREDISEASE",
    "3": "I-RAREDISEASE",
    "4": "B-SIGN",
    "5": "I-SIGN",
    "6": "B-SYMPTOM",
    "7": "I-SYMPTOM",
    "8": "O",
    "9":"O"
  }

In [7]:
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

**Loading relation extraction model**

In [8]:
rel2id={'<pad>':  0,
'produces(e1,e2)':	1,
'anaphora(e1,e2)':	2,
'is_a(e1,e2)':	3,
'is_acron(e1,e2)':	4,
'increases_risk_of(e1,e2)':	5,
'is_synon(e1,e2)':	6,
'produces(e2,e1)':	7,
'anaphora(e2,e1)':	8,
'is_a(e2,e1)':	9,
'is_acron(e2,e1)':	10,
'increases_risk_of(e2,e1)':	11,
'is_synon(e2,e1)':	12}

In [9]:
id2rel={v: k for k, v in rel2id.items()}

In [11]:
import ast
file = open("w2id.txt", "r")

contents = file.read()
word2id = ast.literal_eval(contents)

file.close()

In [12]:
word_vec=torch.load('/drive/My Drive/tensors.pt')

In [13]:
import re
def search_entity(sentence):
    e1 = re.findall(r'<e1>(.*)</e1>', sentence)[0]
    e2 = re.findall(r'<e2>(.*)</e2>', sentence)[0]
    sentence = sentence.replace('<e1>' + e1 + '</e1>', ' <e1> ' + e1 + ' </e1> ', 1)
    sentence = sentence.replace('<e2>' + e2 + '</e2>', ' <e2> ' + e2 + ' </e2> ', 1)
    sentence = sentence.split()
    sentence = ' '.join(sentence)
    sentence = sentence.replace('< e1 >', '<e1>')
    sentence = sentence.replace('< e2 >', '<e2>')
    sentence = sentence.replace('< /e1 >', '</e1>')
    sentence = sentence.replace('< /e2 >', '</e2>')
    sentence = sentence.split()

    assert '<e1>' in sentence
    assert '<e2>' in sentence
    assert '</e1>' in sentence
    assert '</e2>' in sentence

    return sentence

In [14]:
import json
def convert(path_src, path_des):
    with open(path_src, 'r', encoding='utf-8') as fr:
        data = fr.readlines()
        
    with open(path_des, 'w', encoding='utf-8') as fw:
        for i in range(0, len(data), 3):
            id_s, sentence = data[i].strip().split('  ')
            sentence = sentence[1:len(sentence)]
            sentence = search_entity(sentence)
            meta = dict(
                id=id_s,
                relation=data[i+1].strip(),
                sentence=sentence,
                
            )
            json.dump(meta, fw, ensure_ascii=False)
            fw.write('\n')

In [15]:
import numpy as np
class BrevetsDataLoader(object):
    def __init__(self, rel2id, word2id, batch_size, max_len):
        self.rel2id = rel2id
        self.word2id = word2id
        self.batch_size = batch_size
        self.max_len = max_len

    def __collate_fn(self, batch):
        data = zip(*batch)  # unzip the batch data
        data = list(data)
        #label = list(label)
        data = torch.from_numpy(np.concatenate(data, axis=0))
        #label = torch.from_numpy(np.asarray(label, dtype=np.int64))
        return data

    def __get_data(self, filename,  shuffle=False):
        dataset = BrevetsDateset(filename, self.rel2id, self.word2id, self.max_len)
        loader = DataLoader(
            dataset=dataset,
            batch_size= self.batch_size,
            shuffle=shuffle,
            num_workers=2,
            collate_fn=self.__collate_fn
        )
        return loader
    def get_test(self):
        return self.__get_data('sentence.json', shuffle=False)

batch_size = 10
max_len = 90
loader = BrevetsDataLoader(rel2id, word2id, batch_size, max_len)

#test_loader = loader.get_test()

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class Att_BLSTM(nn.Module):
    def __init__(self, word_vec, class_num, max_len, word_dim, hidden_size, layers_num, dropout):
        super().__init__()
        self.word_vec = word_vec
        self.class_num = class_num

        self.max_len = max_len
        self.word_dim = word_dim
        self.hidden_size = hidden_size
        self.layers_num = layers_num
        self.dropout_value = dropout

        self.word_embedding = nn.Embedding.from_pretrained(
            embeddings=self.word_vec,
            freeze=False,
        )
        self.lstm = nn.LSTM(
            input_size=self.word_dim,
            hidden_size=self.hidden_size,
            num_layers=self.layers_num,
            bias=True,
            batch_first=True,
            dropout=self.dropout_value,
            bidirectional=True,
        )
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(self.dropout_value)

        self.att_weight = nn.Parameter(torch.randn(1, self.hidden_size, 1))
        self.dense = nn.Linear(
            in_features=self.hidden_size,
            out_features=self.class_num,
            bias=True
        )
        init.xavier_normal_(self.dense.weight)
        init.constant_(self.dense.bias, 0.)

    def lstm_layer(self, x, mask):
        lengths = torch.sum(mask.gt(0), dim=-1)
        x = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        h, (_, _) = self.lstm(x)
        h, _ = pad_packed_sequence(h, batch_first=True, padding_value=0.0, total_length=self.max_len)
        h = h.view(-1, self.max_len, 2, self.hidden_size)
        h = torch.sum(h, dim=2)  
        return h

    def attention_layer(self, h, mask):
        att_weight = self.att_weight.expand(mask.shape[0], -1, -1) 
        att_score = torch.bmm(self.tanh(h), att_weight)  
        mask = mask.unsqueeze(dim=-1)  
        att_score = att_score.masked_fill(mask.eq(0), float('-inf')) 
        att_weight = F.softmax(att_score, dim=1) 
        reps = torch.bmm(h.transpose(1, 2), att_weight).squeeze(dim=-1) 
        reps = self.tanh(reps)  
        return reps

    def forward(self, data):
        token = data[:, 0, :].contiguous().view(-1, self.max_len)
        mask = data[:, 1, :].contiguous().view(-1, self.max_len)
        emb = self.word_embedding(token) 
        emb = self.dropout(emb)
        h = self.lstm_layer(emb, mask) 
        h = self.dropout(h)
        reps = self.attention_layer(h, mask) 
        reps = self.dropout(reps)
        logits = self.dense(reps)
        return logits

In [17]:
model_re = Att_BLSTM(word_vec = word_vec, class_num = 13, max_len = 90, word_dim = 300, hidden_size = 100, layers_num = 1, dropout = 0.5).to(device)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:57: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [18]:
model_re.load_state_dict(torch.load('/drive/My Drive/tut2-model_ner.pt'))
model_re.eval()

Att_BLSTM(
  (word_embedding): Embedding(400006, 300)
  (lstm): LSTM(300, 100, batch_first=True, dropout=0.5, bidirectional=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.5, inplace=False)
  (dense): Linear(in_features=100, out_features=13, bias=True)
)

In [30]:
def align_tokens_and_predicted_labels(toks_cpu, preds_cpu):
  #print(toks_cpu)
  #print(preds_cpu)
  aligned_toks, aligned_preds = [], []
  prev_tok = None
  for tok, pred in zip(toks_cpu, preds_cpu):
    if tok.startswith("##") and prev_tok is not None:
      prev_tok += tok[2:]
    else:
      if prev_tok is not None:
        aligned_toks.append(prev_tok)
        aligned_preds.append(id2label[str(prev_pred)])
      prev_tok = tok
      prev_pred = pred
  if prev_tok is not None:
    aligned_toks.append(prev_tok)
    aligned_preds.append(id2label[str(prev_pred)])
  return aligned_toks, aligned_preds


def predict(texts):
  aligned_tok_list, aligned_pred_list = [], []
  for text in texts:
    
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model(**inputs)
    tokens_cpu = tokenizer.convert_ids_to_tokens(inputs.input_ids.view(-1))
    preds_cpu = torch.argmax(outputs.logits, dim=-1)[0].cpu().numpy()

    aligned_toks, aligned_preds = align_tokens_and_predicted_labels(tokens_cpu, preds_cpu)

    aligned_tok_list.append(aligned_toks)
    aligned_pred_list.append(aligned_preds)

  return aligned_tok_list, aligned_pred_list


### After loading best ner & re models , let's apply them to our corpus

In [20]:
import glob
path='/drive/MyDrive/corpus'
all_files = glob.glob(path + "/*")

In [21]:
text_files=glob.glob(path+"/*.txt")
for x in text_files:
      print(x)
      break

/drive/MyDrive/corpus/WAS-Related-Disorders.txt


- read each file
- get sentences
- extract entities from each sentence using ner model
- if a sentence got two entities : surround them with la balise e then apply relation extraction model 
- store in a dataframe the following informations : "entity1" , "tag1", "relation_label","entity2","tag2"
- create a kg using py2neo


In [22]:
source=[]
target=[]
relation=[]
tags1=[]
tags2=[]

In [23]:
def get_prediction(model, iterator):
  predict_label=[]
  for _, data in enumerate(iterator):
            data = data.to(device)

            logits = model(data)
            

            _, pred = torch.max(logits, dim=1)  
            pred = pred.cpu().detach().numpy().reshape((-1, 1))
            predict_label.append(pred)
            predict_label = np.concatenate(predict_label, axis=0).reshape(-1).astype(np.int64)
  return(predict_label)

In [24]:
import os
import json
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [25]:
class BrevetsDateset(Dataset):
    def __init__(self, filename, rel2id, word2id, max_len):
        self.filename = filename
        self.rel2id = rel2id
        self.word2id = word2id
        self.max_len = max_len
        self.data_dir = ''
        self.dataset = self.__load_data()

    def __symbolize_sentence(self, sentence):
      
        mask = [1] * len(sentence)
        words = []
        length = min(self.max_len, len(sentence))
        mask = mask[:length]

        for i in range(length):
            words.append(self.word2id.get(sentence[i].lower(), self.word2id['UNK']))

        if length < self.max_len:
            for i in range(length, self.max_len):
                mask.append(0)  # 'PAD' mask is zero
                words.append(self.word2id['PAD'])

        unit = np.asarray([words, mask], dtype=np.int64)
        unit = np.reshape(unit, newshape=(1, 2, self.max_len))
        return unit

    def __load_data(self):
        path_data_file = os.path.join(self.data_dir, self.filename)
        data = []
        #labels = []
        with open(path_data_file, 'r', encoding='utf-8') as fr:
            for line in fr:
                line = json.loads(line.strip())
                #label = line['relation']
                sentence = line['sentence']
                #label_idx = self.rel2id[label]

                one_sentence = self.__symbolize_sentence(sentence)
                data.append(one_sentence)
                #labels.append(label_idx)
        return data

    def __getitem__(self, index):
        index=0
        data = self.dataset[index]
        #label = self.label[index]
        return data
    def __len__(self):
        return len('relation')

In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
# remove extra spaces and ending space if any
import re
spaces = ['\u200b', '\u200e', '\u202a', '\u202c', '\ufeff', '\uf0d8', '\u2061', '\x10', '\x7f', '\x9d', '\xad', '\xa0']
def remove_space(text):
    for space in spaces:
        text = text.replace(space, ' ')
    text = text.strip()
    text = re.sub('\s+', ' ', text)
    return text

In [31]:
id=0
for file in text_files:
    with open(file, 'r', encoding = 'utf-8') as f:
        text = f.read()
    text=text.replace('\n',' ')
    text=remove_space(text)
    sentences=tokenize.sent_tokenize(text)
    sentences=[[s[:512]] for s in sentences] # I returned it this form so that i can apply ner model on each sentence  
    predicted_tokens, predicted_tags = predict(sentences)
    for i in range (len(sentences)):
      firstTag=True
      sentence=""
      other=['O' for o in range (len(predicted_tags[i]))] # sentence which doesnt contain any tag
      if predicted_tags[i] != other: 
        for tok in range (len(predicted_tokens[i])):
          if predicted_tags[i][tok]=='O':
            sentence=sentence+' '+predicted_tokens[i][tok]
          elif predicted_tokens[i][tok]!='[SEP]' and predicted_tags[i][tok].startswith('B-') and firstTag:
            tag1=predicted_tags[i][tok][2:]
            if tok!=(len(predicted_tags[i])-1) and predicted_tags[i][tok+1].startswith('I-'):
              sentence=sentence+' '+'<e1>'+predicted_tokens[i][tok]
            else:
              sentence=sentence+' '+'<e1>'+predicted_tokens[i][tok]+'</e1>'
            firstTag=False
          elif predicted_tokens[i][tok]!='[SEP]' and predicted_tags[i][tok].startswith('B-') and firstTag is False:
            tag2=predicted_tags[i][tok][2:]
            if tok!=(len(predicted_tags[i])-1) and predicted_tags[i][tok+1].startswith('I-') and predicted_tokens[i][tok+1]!='[SEP]':
              sentence=sentence+' '+'<e2>'+predicted_tokens[i][tok]
            else:
              sentence=sentence+' '+'<e2>'+predicted_tokens[i][tok]+'</e2>'
          else:
            if predicted_tokens[i][tok]!='[SEP]':
              if tok!=(len(predicted_tags[i])-1) and predicted_tags[i][tok+1].startswith('I-') and predicted_tokens[i][tok+1]!='[SEP]' :
                 sentence=sentence+' '+predicted_tokens[i][tok]
              else:
                 if "<e2>" in sentence:
                   sentence=sentence+' '+predicted_tokens[i][tok]+"</e2>"
                 else:
                   sentence=sentence+' '+predicted_tokens[i][tok]+'</e1>'


      if sentence.count("<e1>")==1 and sentence.count("<e2>")==1:
        m=sentence.find('<e1>')  
        n=sentence.find('</e1>')
        k=m+4
        entity1=sentence[k:n] 
        p= sentence.find('<e2>')
        q=sentence.find('</e2>')
        y=p+4
        entity2=sentence[y:q]
        #write sentence in text file
        with open('sentence.txt', 'w') as f:
           id=1
           f.write(str(id)+'  '+sentence)
           f.write("\n")
           f.write('relation')
        f.close()
        convert('sentence.txt','sentence.json')
        test_loader = loader.get_test()
        pred=get_prediction(model_re, test_loader)
        rel=id2rel[pred[0]]
        if "(e1,e2)" in rel:
          source.append(entity1)
          target.append(entity2)
          relation.append(rel[:-7])
          tags1.append(tag1)
          tags2.append(tag2)
        else:
          source.append(entity2)
          target.append(entity1)
          relation.append(rel[:-7])
          tags1.append(tag2)
          tags2.append(tag1)


In [32]:
data2=pd.DataFrame({
    'source':source,
    'tag1':tags1,
    'relation_label':relation,
    'target':target,
    'tag2':tags2
})

In [34]:
data2

,source,tag1,relation_label,target,tag2
0,x - linked congenital neutropenia,RAREDISEASE,is_acron,wiskott - aldrich syndrome,RAREDISEASE
1,[CLS],DISEASE,produces,bleeding,DISEASE
2,,RAREDISEASE,produces,to adolescence,DISEASE
3,wolfram syndrome affects males,RAREDISEASE,anaphora,and females in equal numbers,DISEASE
4,weil syndrome,RAREDISEASE,produces,leptospirosis,RAREDISEASE
...,...,...,...,...,...
752,heart,RAREDISEASE,produces,defects are present,DISEASE
753,,SIGN,is_a,widespread symptoms,SIGN
754,in,SIGN,produces,be,DISEASE
755,,RAREDISEASE,produces,the,SIGN


In [ ]:
df.to_csv('data2.csv')